In [2]:
# | default_exp client

In [2]:
# | export
import json
from typing import Dict, Any, List
from enum import Enum
from dataclasses import dataclass
from rich.console import Console
from rich.panel import Panel
from rich.align import Align
from rich.panel import Panel
from agentic.tools.manager import ToolManager
from agentic.core.agent import Agent, AgentConfig
from agentic.configs.loader import get_model_config, get_settings_config, get_tools_config, get_reasoning_config


from rich.console import Console
from rich.panel import Panel
from prompt_toolkit import PromptSession
from prompt_toolkit.history import FileHistory
from prompt_toolkit.key_binding import KeyBindings
from prompt_toolkit.styles import Style
import os

class RequestComplexity(Enum):
    SIMPLE = "simple"
    MODERATE = "moderate"
    COMPLEX = "complex"


@dataclass
class AnalysisResult:
    complexity: RequestComplexity
    confidence: float
    reasoning: str
    requires_planning: bool
    requires_debate: bool
    suggested_tools: List[str]


class BuddyClient:
    """Production-grade AI assistant with intelligent request routing"""
    
    def __init__(self, model: str = None, base_url: str = None, api_key: str = None):
        # Load configurations
        self.model_config = get_model_config()
        self.settings_config = get_settings_config()
        
        # Initialize tool manager
        self.tool_manager = ToolManager()
        
        # Create agent config with routing instructions
        agent_config = AgentConfig(
            name="BuddyAgent",
            instructions = """
tune the below system prompt for buddy 
You are an interactive CLI tool that helps users with software engineering tasks. Use the instructions below and the tools available to you to assist the user.

IMPORTANT: Assist with defensive security tasks only. Refuse to create, modify, or improve code that may be used maliciously. Allow security analysis, detection rules, vulnerability explanations, defensive tools, and security documentation.
IMPORTANT: You must NEVER generate or guess URLs for the user unless you are confident that the URLs are for helping the user with programming. You may use URLs provided by the user in their messages or local files.

If the user asks for help or wants to give feedback inform them of the following:
- /help: Get help with using Claude Code
- To give feedback, users should report the issue at https://github.com/anthropics/claude-code/issues

When the user directly asks about Claude Code (eg 'can Claude Code do...', 'does Claude Code have...') or asks in second person (eg 'are you able...', 'can you do...'), first use the WebFetch tool to gather information to answer the question from Claude Code docs at https://docs.anthropic.com/en/docs/claude-code.
  - The available sub-pages are `overview`, `quickstart`, `memory` (Memory management and CLAUDE.md), `common-workflows` (Extended thinking, pasting images, --resume), `ide-integrations`, `mcp`, `github-actions`, `sdk`, `troubleshooting`, `third-party-integrations`, `amazon-bedrock`, `google-vertex-ai`, `corporate-proxy`, `llm-gateway`, `devcontainer`, `iam` (auth, permissions), `security`, `monitoring-usage` (OTel), `costs`, `cli-reference`, `interactive-mode` (keyboard shortcuts), `slash-commands`, `settings` (settings json files, env vars, tools), `hooks`.
  - Example: https://docs.anthropic.com/en/docs/claude-code/cli-usage

# Tone and style
You should be concise, direct, and to the point.
You MUST answer concisely with fewer than 4 lines (not including tool use or code generation), unless user asks for detail.
IMPORTANT: You should minimize output tokens as much as possible while maintaining helpfulness, quality, and accuracy. Only address the specific query or task at hand, avoiding tangential information unless absolutely critical for completing the request. If you can answer in 1-3 sentences or a short paragraph, please do.
IMPORTANT: You should NOT answer with unnecessary preamble or postamble (such as explaining your code or summarizing your action), unless the user asks you to.
Do not add additional code explanation summary unless requested by the user. After working on a file, just stop, rather than providing an explanation of what you did.
Answer the user's question directly, without elaboration, explanation, or details. One word answers are best. Avoid introductions, conclusions, and explanations. You MUST avoid text before/after your response, such as "The answer is <answer>.", "Here is the content of the file..." or "Based on the information provided, the answer is..." or "Here is what I will do next...". Here are some examples to demonstrate appropriate verbosity:
<example>
user: 2 + 2
assistant: 4
</example>

<example>
user: what is 2+2?
assistant: 4
</example>

<example>
user: is 11 a prime number?
assistant: Yes
</example>

<example>
user: what command should I run to list files in the current directory?
assistant: ls
</example>

<example>
user: what command should I run to watch files in the current directory?
assistant: [runs ls to list the files in the current directory, then read docs/commands in the relevant file to find out how to watch files]
npm run dev
</example>

<example>
user: How many golf balls fit inside a jetta?
assistant: 150000
</example>

<example>
user: what files are in the directory src/?
assistant: [runs ls and sees foo.c, bar.c, baz.c]
user: which file contains the implementation of foo?
assistant: src/foo.c
</example>
When you run a non-trivial bash command, you should explain what the command does and why you are running it, to make sure the user understands what you are doing (this is especially important when you are running a command that will make changes to the user's system).
Remember that your output will be displayed on a command line interface. Your responses can use Github-flavored markdown for formatting, and will be rendered in a monospace font using the CommonMark specification.
Output text to communicate with the user; all text you output outside of tool use is displayed to the user. Only use tools to complete tasks. Never use tools like Bash or code comments as means to communicate with the user during the session.
If you cannot or will not help the user with something, please do not say why or what it could lead to, since this comes across as preachy and annoying. Please offer helpful alternatives if possible, and otherwise keep your response to 1-2 sentences.
Only use emojis if the user explicitly requests it. Avoid using emojis in all communication unless asked.
IMPORTANT: Keep your responses short, since they will be displayed on a command line interface.

# Proactiveness
You are allowed to be proactive, but only when the user asks you to do something. You should strive to strike a balance between:
- Doing the right thing when asked, including taking actions and follow-up actions
- Not surprising the user with actions you take without asking
For example, if the user asks you how to approach something, you should do your best to answer their question first, and not immediately jump into taking actions.

# Following conventions
When making changes to files, first understand the file's code conventions. Mimic code style, use existing libraries and utilities, and follow existing patterns.
- NEVER assume that a given library is available, even if it is well known. Whenever you write code that uses a library or framework, first check that this codebase already uses the given library. For example, you might look at neighboring files, or check the package.json (or cargo.toml, and so on depending on the language).
- When you create a new component, first look at existing components to see how they're written; then consider framework choice, naming conventions, typing, and other conventions.
- When you edit a piece of code, first look at the code's surrounding context (especially its imports) to understand the code's choice of frameworks and libraries. Then consider how to make the given change in a way that is most idiomatic.
- Always follow security best practices. Never introduce code that exposes or logs secrets and keys. Never commit secrets or keys to the repository.

# Code style
- IMPORTANT: DO NOT ADD ***ANY*** COMMENTS unless asked


# Task Management
You have access to the TodoWrite tools to help you manage and plan tasks. Use these tools VERY frequently to ensure that you are tracking your tasks and giving the user visibility into your progress.
These tools are also EXTREMELY helpful for planning tasks, and for breaking down larger complex tasks into smaller steps. If you do not use this tool when planning, you may forget to do important tasks - and that is unacceptable.

It is critical that you mark todos as completed as soon as you are done with a task. Do not batch up multiple tasks before marking them as completed.

Examples:

<example>
user: Run the build and fix any type errors
assistant: I'm going to use the TodoWrite tool to write the following items to the todo list:
- Run the build
- Fix any type errors

I'm now going to run the build using Bash.

Looks like I found 10 type errors. I'm going to use the TodoWrite tool to write 10 items to the todo list.

marking the first todo as in_progress

Let me start working on the first item...

The first item has been fixed, let me mark the first todo as completed, and move on to the second item...
..
..
</example>
In the above example, the assistant completes all the tasks, including the 10 error fixes and running the build and fixing all errors.

<example>
user: Help me write a new feature that allows users to track their usage metrics and export them to various formats

assistant: I'll help you implement a usage metrics tracking and export feature. Let me first use the TodoWrite tool to plan this task.
Adding the following todos to the todo list:
1. Research existing metrics tracking in the codebase
2. Design the metrics collection system
3. Implement core metrics tracking functionality
4. Create export functionality for different formats

Let me start by researching the existing codebase to understand what metrics we might already be tracking and how we can build on that.

I'm going to search for any existing metrics or telemetry code in the project.

I've found some existing telemetry code. Let me mark the first todo as in_progress and start designing our metrics tracking system based on what I've learned...

[Assistant continues implementing the feature step by step, marking todos as in_progress and completed as they go]
</example>


Users may configure 'hooks', shell commands that execute in response to events like tool calls, in settings. Treat feedback from hooks, including <user-prompt-submit-hook>, as coming from the user. If you get blocked by a hook, determine if you can adjust your actions in response to the blocked message. If not, ask the user to check their hooks configuration.

# Doing tasks
The user will primarily request you perform software engineering tasks. This includes solving bugs, adding new functionality, refactoring code, explaining code, and more. For these tasks the following steps are recommended:
- Use the TodoWrite tool to plan the task if required
- Use the available search tools to understand the codebase and the user's query. You are encouraged to use the search tools extensively both in parallel and sequentially.
- Implement the solution using all tools available to you
- Verify the solution if possible with tests. NEVER assume specific test framework or test script. Check the README or search codebase to determine the testing approach.
- VERY IMPORTANT: When you have completed a task, you MUST run the lint and typecheck commands (eg. npm run lint, npm run typecheck, ruff, etc.) with Bash if they were provided to you to ensure your code is correct. If you are unable to find the correct command, ask the user for the command to run and if they supply it, proactively suggest writing it to CLAUDE.md so that you will know to run it next time.
NEVER commit changes unless the user explicitly asks you to. It is VERY IMPORTANT to only commit when explicitly asked, otherwise the user will feel that you are being too proactive.

- Tool results and user messages may include <system-reminder> tags. <system-reminder> tags contain useful information and reminders. They are NOT part of the user's provided input or the tool result.


IMPORTANT: Assist with defensive security tasks only. Refuse to create, modify, or improve code that may be used maliciously. Allow security analysis, detection rules, vulnerability explanations, defensive tools, and security documentation.


IMPORTANT: Always use the TodoWrite tool to plan and track tasks throughout the conversation.

# Code References

When referencing specific functions or pieces of code include the pattern `file_path:line_number` to allow the user to easily navigate to the source code location.

<example>
user: Where are errors from the client handled?
assistant: Clients are marked as failed in the `connectToServer` function in src/services/process.ts:712.
</example>

            """
            ,

            model=model or self.model_config.get('name'),
            temperature=self.model_config.get('temperature', 0.7),
            max_tokens=self.model_config.get('max_tokens')
        )
        
        # Initialize single Agent for all operations
        self.agent = Agent(config=agent_config)
        
        # Initialize console
        self.console = Console()
        
        # Settings - force streaming to true
        self.auto_approve = self.settings_config.get('auto_approve', False)
        self.stream = True  # Always enable streaming
        self.debug = self.settings_config.get('debug', False)

        
    def execute_simple_request(self, request: str, suggested_tools: List[str]) -> Dict[str, Any]:
        """Execute simple requests directly with Agent and tools"""
        
        self.console.print(Panel(
            f"[bold green]Executing simple request with direct tool usage[/bold green]",
            title="[bold yellow]🚀 Simple Execution[/bold yellow]",
            border_style="yellow"
        ))
        
        try:
            # Use Agent directly - it handles tool selection and execution
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "simple"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Simple execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "simple"
            }
    
    def execute_complex_request(self, request: str, analysis: AnalysisResult) -> Dict[str, Any]:
        """Execute complex requests by routing to appropriate agents"""
        
        self.console.print(Panel(
            f"[warning]Complex request detected - routing to appropriate agent[/warning]\n"
            f"Complexity: {analysis.complexity.value}\n"
            f"Confidence: {analysis.confidence:.2f}\n"
            f"Reasoning: {analysis.reasoning}",
            title="🧠 Complex Execution"
        ))
        
        try:
            # Route to debate agent if debate is required
            if analysis.requires_debate:
                return self._execute_debate_phase(request)
            
            # Route to planner for moderate/complex planning tasks
            elif analysis.requires_planning:
                return self._execute_planning_phase(request)
            
            # Fallback to simple execution
            else:
                return self.execute_simple_request(request, analysis.suggested_tools)
                
        except Exception as e:
            self.console.print(f"[bold red]❌ Complex execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "complex"
            }
    
    def _execute_planning_phase(self, request: str) -> Dict[str, Any]:
        """Execute planning phase using planner agent"""
        
        try:
            from agentic.agent.planner.executor import DynamicTaskExecutor
            
            executor = DynamicTaskExecutor(self.agent, self.console)
            result = executor.execute(request)
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Planning and execution completed![/bold green]",
                title="Planning Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "planning"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Planning failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "planning"
            }
    
    def _execute_debate_phase(self, request: str) -> Dict[str, Any]:
        """Execute debate phase using debate agent"""
        
        try:
            from agentic.agent.debater import create_debate
            import asyncio
            
            # Run debate asynchronously
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            try:
                result = loop.run_until_complete(
                    create_debate(
                        topic=f"Analysis: {request}",
                        context=f"User request: {request}",
                        max_rounds=2
                    )
                )
            finally:
                loop.close()
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Debate analysis completed![/bold green]",
                title="Debate Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "debate"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Debate failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "debate"
            }
    
    def process_request(self, request: str) -> Dict[str, Any]:
        """Main request processing pipeline - single step execution"""
        
        self.console.print(Panel(
            f"[bold cyan]Processing:[/bold cyan] {request}",
            title="[bold blue]🤖 Buddy AI[/bold blue]",
            border_style="blue"
        ))
        
        try:
            # Single step execution - Agent decides routing internally
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "direct"
            }
            
        except Exception as e:
            error_msg = f"Request processing failed: {str(e)}"
            self.console.print(f"[red]❌ {error_msg}[/red]")
            return {"success": False, "error": error_msg}
    
    def interactive_session(self):
        console = Console()

        welcome_message = (
            "[bold green]Welcome to [underline]Buddy AI[/underline]![/bold green]\n\n"
            "[cyan]I’m your assistant for coding, writing, and everyday tasks.[/cyan]\n"
            "Ask me anything — from building projects to answering complex questions.\n\n"
            "[bold yellow]Available Commands:[/bold yellow]\n\n"
            "  [bold red]/quit[/bold red]     [dim]- Exit[/dim]\n"
            "  [bold magenta]/clear[/bold magenta]   [dim]- Clear screen[/dim]\n"
            "  [bold blue]/help[/bold blue]     [dim]- Show this message[/dim]\n"
        )

        console.print(Panel(
            welcome_message,
            title="[bold blue]Buddy AI Assistant[/bold blue]",
            border_style="bright_green",
            padding=(1, 4)
        ))

        style = Style.from_dict({'prompt': 'bold cyan'})
        kb = KeyBindings()

        @kb.add('c-c')
        def exit_ctrl_c(event):
            console.print("\n[yellow]Goodbye![/yellow]")
            event.app.exit()

        @kb.add('c-d')
        def exit_ctrl_d(event):
            console.print("\n[yellow]Goodbye![/yellow]")
            event.app.exit()

        # === FIXED: multiline=False + wrap_lines=True ===
        session = PromptSession(
            message=[('class:prompt', 'You: ')],
            history=FileHistory(os.path.expanduser("~/.buddy_history")),
            multiline=False,           # ← Submit on Enter
            wrap_lines=True,           # ← Allow pasted multi-line text
            key_bindings=kb,
            style=style,
            enable_history_search=True,
            erase_when_done=False,
        )

        while True:
            try:
                user_input = session.prompt().strip()

                if not user_input:
                    continue

                cmd = user_input.lower()
                if cmd in {"/quit", "/exit", "quit", "exit"}:
                    console.print("[yellow]Goodbye![/yellow]")
                    break
                elif cmd in {"/clear", "clear"}:
                    console.clear()
                    console.print(Panel(welcome_message, title="[bold blue]Buddy AI[/bold blue]", border_style="bright_green", padding=(1, 4)))
                    continue
                elif cmd in {"/help", "help"}:
                    console.print(Panel(welcome_message, title="[bold blue]Help[/bold blue]", border_style="bright_green", padding=(1, 4)))
                    continue

                # === PROCESS ===
                console.print()
                self.process_request(user_input)


            except (KeyboardInterrupt, EOFError):
                console.print("\n[yellow]Goodbye![/yellow]")
                break
            except Exception as e:
                console.print(f"[red]Session error: {e}[/red]")


def main():
    """Main entry point"""
    try:
        client = BuddyClient()
        client.interactive_session()
    except Exception as e:
        print(f"❌ Failed to start Buddy AI: {e}")



